In [ ]:
from FLUOstar_main import *
from tkinter import filedialog as fd
from tkinter import *
from tkinter import ttk
import tkinter as tk

################################################################################################################################

## full sequence 
active_program = True

while active_program:

    # retrieve all documents
    all_docs_dict = document_retrieval()
    all_docs_keys = all_docs_dict.keys()
    all_docs_dict_preprocessed = {}

    # add documents to dictionary with all docs preprocessed
    for i in all_docs_keys:
        current_doc_add = all_docs_dict[i]

        try:
            current_doc_db = pd.read_excel(current_doc_add)
        except FileNotFoundError:
            print("No file was found.")
        
        all_docs_dict_preprocessed[i] = preprocessing(current_doc_db)

    # check what is this truly doing
    all_docs_final_dict = time_processing_doc()
    all_docs_list = concatenate_docs()

    # create full docs with all docs all preprocessed
    full_doc = pd.concat(all_docs_list)

    # ask and create conditions groups with replicates + define wells/columns to be kept
    conditions_groups = create_groups()
    wells_to_keep = get_wells_to_keep(conditions_groups)

    # eliminate unused wells from full docs
    full_doc = drop_unused_wells(full_doc, wells_to_keep)

    # show plots for all conditions using all wells
    create_first_plots()
    plt.show()

    # ask user if any wells will be deleted and update full doc df + condition group dict accordingly
    wells_to_keep_updated = deleting_wells(wells_to_keep)
    full_doc = drop_unused_wells(full_doc, wells_to_keep_updated)
    conditions_groups_up = conditions_groups.copy()
    conditions_groups_up = update_conditions_groups(conditions_groups_up)

    # change name of full doc creating a new updated df
    full_doc_good_names = change_column_names(full_doc)

    # generate df with average for all conditions
    averages_df = averages_df_gen(full_doc_good_names)

    # generate plot with averages
    average_plot = averages_df.plot()
    plt.show()

    # generate df with standar deviations for all conditions
    stdev_df = stdev_df_gen(full_doc_good_names)

    # generate plot with averages + std deviations
    average_plot_sd = averages_df.plot(yerr=stdev_df)

    # generate df with both averages and standar deviations
    averages_std_df = merge_ave_std(averages_df, stdev_df)

    # ask user input for initial and final times for normalization of each replicate, put info in a dictionary
    times_norm_dic = get_norm_times(full_doc)

    # generate df with normalized data
    norm_df = full_doc.copy()
    norm_df = norm_df.reset_index(drop=True)
    norm_df = normalization(norm_df)
    norm_df = change_column_names(norm_df)

    # generate df with averages of normalized data
    norm_averages_df = averages_df_gen(norm_df)
    # generate df with standar deviations of normalized data
    norm_stdev_df = stdev_df_gen(norm_df)

    # plot normalized data (averages + stdev)
    norm_average_plot = norm_averages_df.plot(yerr=norm_stdev_df)
    plt.show()

    # generates df with averages + stdevs of normalized data
    norm_ave_std_df = merge_ave_std(norm_averages_df, norm_stdev_df)

    # check what is happ here---
    norm_std_ave_df = averages_df_gen(norm_stdev_df)


    # calculate half-times + generate df with them
    half_times = half_times_df(norm_df)
    half_times = half_times.reset_index()
    half_times = half_times.rename(columns={half_times.columns[0]: 'Condition', half_times.columns[1]: 'half-times'})

    # calculate and generate averages + std devs for half times
    half_times_ave = half_times.groupby(by='Condition').mean()
    half_times_stdev = half_times.groupby(by='Condition').std()
    half_times_stdev = half_times_stdev.rename(columns={half_times.columns[1]: 'half-times stdev'})
    ht_std_column = half_times_stdev['half-times stdev']
    half_times_ave_std = half_times_ave.join(ht_std_column)

    # generate final excel file
    final_doc_name = input('Name for the final Excel document: ')
    final_doc_dir = fd.askdirectory()
    root.withdraw()
    final_dir = final_doc_dir + "/" + final_doc_name + ".xlsx"

    with pd.ExcelWriter(final_dir) as writer:
        full_doc_good_names.to_excel(writer, sheet_name="Full")  
        averages_std_df.to_excel(writer, sheet_name="Averages")  
        norm_df.to_excel(writer, sheet_name="Normalized")  
        norm_ave_std_df.to_excel(writer, sheet_name="Normalized (ave)")
        half_times.to_excel(writer, sheet_name="half-times")
        half_times_ave_std.to_excel(writer, sheet_name="half-times (ave)")
        full_doc.to_excel(writer, sheet_name="Full data")
    
    # destroy tkinter window
    root.destroy()

    root.mainloop()
    active_program = False